In [ ]:
from flask import Flask
import time as tt
import datetime
import calendar

from werkzeug.serving import run_simple

import requests
import json
import csv
import pandas as pd
import json
from pandas.io.json import json_normalize
import re
from pygeocoder import Geocoder
import pycountry
import flatdict
import pandas as pd
import arcgis
from arcgis.gis import GIS
from IPython.display import display

app = Flask(__name__)

@app.route('/display/<latitude>,<longitude>,<time>,<param>')

def display(latitude,longitude,time,param):
    '''
    print('--------------------------')
    print(latitude)
    print('--------------------------')
    print(longitude)
    print('--------------------------')

    '''
    string="https://api.darksky.net/forecast/68d70a9cf38161567cc4aec124be92ed/"+latitude+","+longitude+","+time+"?exclude=hourly,currently,minutely,alert,flags"

    response = requests.get(string)
    data=response.content
    if(len(data)<140):
        return('light not available')
    parsed_data=pd.read_json(data.decode('utf-8'))

    new=parsed_data['daily'][0]
    #print(new)
    new1=pd.DataFrame(new)
    latl=[]
    longl=[]
    for i in range(len(new1)):
        latl.append(latitude)
        longl.append(longitude)
    new1['lat']=latl
    new1['long']=longl
    #print(new1)

    gis=GIS()
    map1 = gis.map("US", zoomlevel = 8)
    
    #df = pd.read_csv('newww.csv')
    df=new1
    light = gis.content.import_data(df)
    light.layer.layers = [dict(light.layer)]
    map1.add_layer(light)
    try1=light.query()
    dfn=((try1).df)
    del dfn['SHAPE']
    #print(dfn)
    js=dfn.to_json(orient='index')
    if(param=="light"):
        return (js)
    #print("light pols")
    
    #air
   
    response = requests.get("https://api.openaq.org/v1/measurements?coordinates="+latitude+","+longitude+"&radius=28000"+"&parameter=pm25")

    variable=str(response.content)
    if(len(variable)<140):
        return('air not available')
    xizz=variable[2:-1].encode('utf-8')
    xiz=xizz.decode('utf-8')
    st1=re.sub('"unit":".*?",', '', xiz)
    #st2=re.sub('"city":".*?",', '', st1)
    st=re.sub(',"city":".*?"}', '}', st1)

    data = json.loads(st)
    #print(st)
    df = json_normalize(data, 'results')

    dict1=list(df['coordinates'])
    #print(dict1)
    df2=pd.DataFrame(dict1)
    #print(df2)
    df['lat']=df2['latitude']
    df['long']=df2['longitude']

   
    del df['coordinates']

    #print(df['date'])
    dflist = df['date'].tolist()
    #print(dfList)
    l_data=[]
    l_local=[]
    for i in dflist:
        #print (i['local'][:10])
        l_data.append(i['local'][:10])
    for i in dflist:
        #print (i['local'][20:])
        l_local.append(i['local'][20:])
    #print(l_local)
    #print(l_data)
    del df['date']
    del df['location']
    df4 = pd.DataFrame()
    df4['dates']=l_data
    df4['local']=l_local
    df['date']=df4['dates']
    df['local']=df4['local']
    epoch=[]
    '''for i in range(len(df)):
        date_time = df['date'][i]
        pattern = '%Y.%m.%d'
        utc_epoch =  calendar.timegm(time.strftime(date_time, pattern))
        print(utc_epoch+"\n")
        epoch.append(utc_epoch)
    '''
    for i in range(len(df)):
        date_time = df['date'][i]
        t = int(tt.mktime(tt.strptime(str(df['date'][i]), "%Y-%m-%d")))
        epoch.append(t)
    df['epocht']=epoch
    #print(df)
    df.to_csv("final.csv")


    gis=GIS()
    map1 = gis.map("Amsterdam", zoomlevel = 8)


    df = pd.read_csv('final.csv')
    airpol = gis.content.import_data(df)
    airpol.layer.layers = [dict(airpol.layer)]
    map1.add_layer(airpol)
    try1=airpol.query().df
    del try1['SHAPE']
    del try1['Unnamed__0']
    time1=0
    #print(try1)
    #print("\n")
    #print(time)
    #print("\n")
    time1=int(time)
    min1=99999999999
    key1=0
    #print(time1)
    #print("\n")
    for i in range(len(try1['epocht'])):
        if((abs(int(try1['epocht'][i])-time1))<=min1):
            min1=abs(int(try1['epocht'][i])-time1)
            key1=i
            if(min1==0):
                break
    #print(key1)
    try2=try1.loc[key1]
    #print(try1)
    js2=try2.to_json(orient='index')
    if(param=="air"):
        return(js2)
    
    #viirs
    '''
    viirs=pd.read_csv('E:/UCI-Research/VNFcopy.csv')
    virlight=gis.content.import_data(viirs)
    virlight.layer.layers = [dict(virlight.layer)]
    map1.add_layer(virlight)
    virdf=virlight.query()
    dfn=((virdf).df.head())
    #try2=dfn.loc[key1]
    #print(try1)
    js3=dfn.to_json(orient='index')
    if(param=='viirs'):
        return(js3)
    '''
    #print("air pols")
    #return "Looks like it works!"
    if(param=="lightair"):
        return(js+js2)
    elif(param=="airlight"):
        return(js2+js)
if __name__=='__main__':
	run_simple('localhost', 9005, app)

In [1]:
from flask import Flask
import time as tt
import datetime
import calendar

from werkzeug.serving import run_simple

import requests
import json
import csv
import pandas as pd
import json
from pandas.io.json import json_normalize
import re
from pygeocoder import Geocoder
import pycountry
import flatdict
import pandas as pd
import arcgis
from arcgis.gis import GIS
from IPython.display import display
from bs4 import BeautifulSoup
from urllib.request import Request, urlopen
import re
import gzip
import shutil
import requests
app = Flask(__name__)

@app.route('/display/<latitude>,<longitude>,<time>,<param>')

def display(latitude,longitude,time,param):
    '''
    print('--------------------------')
    print(latitude)
    print('--------------------------')
    print(longitude)
    print('--------------------------')

    '''
    def Convert(tup, dic):
        for a, b in tup:
            dic.setdefault(a, []).append(b)
        return dic
    string="https://api.darksky.net/forecast/68d70a9cf38161567cc4aec124be92ed/"+latitude+","+longitude+","+time+"?exclude=hourly,currently,minutely,alert,flags"

    response = requests.get(string)
    data=response.content
    if(len(data)<140):
        return('light not available')
    parsed_data=pd.read_json(data.decode('utf-8'))

    new=parsed_data['daily'][0]
    new1=pd.DataFrame(new)
    latl=[]
    longl=[]
    for i in range(len(new1)):
        latl.append(latitude)
        longl.append(longitude)
    new1['lat']=latl
    new1['long']=longl

    gis=GIS()
    map1 = gis.map("US", zoomlevel = 8)
    df=new1
    light = gis.content.import_data(df)
    light.layer.layers = [dict(light.layer)]
    map1.add_layer(light)
    try1=light.query()
    dfn=((try1).df)
    del dfn['SHAPE']
    js=dfn.to_json(orient='index')
    parsed = json.loads(js)
    di={}
    if(param=="light"):
        new_list=[]
        for key,value in parsed["0"].items():
            if(key!="lat" and key!="long"):
                new_list.append((key,value))
        Convert(new_list, di)
        for k in list(parsed["0"].keys()):
            if k != 'lat' and k != 'long':
                del parsed["0"][k]

        parsed["lat"]=parsed["0"]["lat"]
        parsed["long"]=parsed["0"]["long"]

        del parsed["0"]
        parsed['values']=di
        lightt= (json.dumps(parsed, indent=4, sort_keys=True))
        return(lightt)
    
    #air
    response = requests.get("https://api.openaq.org/v1/measurements?coordinates="+latitude+","+longitude+"&radius=28000"+"&parameter=pm25")
    variable=str(response.content)
    if(len(variable)<140):
        return('air not available')
    xizz=variable[2:-1].encode('utf-8')
    xiz=xizz.decode('utf-8')
    st1=re.sub('"unit":".*?",', '', xiz)
    st=re.sub(',"city":".*?"}', '}', st1)
    data = json.loads(st)
    df = json_normalize(data, 'results')
    dict1=list(df['coordinates'])
    df2=pd.DataFrame(dict1)
    df['lat']=df2['latitude']
    df['long']=df2['longitude']
    del df['coordinates']
    dflist = df['date'].tolist()
    l_data=[]
    l_local=[]
    for i in dflist:
        l_data.append(i['local'][:10])
    for i in dflist:
        l_local.append(i['local'][20:])
    del df['date']
    del df['location']
    df4 = pd.DataFrame()
    df4['dates']=l_data
    df4['local']=l_local
    df['date']=df4['dates']
    df['local']=df4['local']
    epoch=[]
    for i in range(len(df)):
        date_time = df['date'][i]
        t = int(tt.mktime(tt.strptime(str(df['date'][i]), "%Y-%m-%d")))
        epoch.append(t)
    df['epocht']=epoch
    df.to_csv("final.csv")
    gis=GIS()
    map1 = gis.map("Amsterdam", zoomlevel = 8)
    df = pd.read_csv('final.csv')
    airpol = gis.content.import_data(df)
    airpol.layer.layers = [dict(airpol.layer)]
    map1.add_layer(airpol)
    try1=airpol.query().df
    del try1['SHAPE']
    del try1['Unnamed__0']
    time1=0
    time1=int(time)
    min1=99999999999
    key1=0
    for i in range(len(try1['epocht'])):
        if((abs(int(try1['epocht'][i])-time1))<=min1):
            min1=abs(int(try1['epocht'][i])-time1)
            key1=i
            if(min1==0):
                break
    try2=try1.loc[key1]
    js2=try2.to_json(orient='index')
    parsed2=json.loads(js2)
    if(param=="air"):
        new_list2=[(parsed2['parameter'],parsed2['value'])]
        di2= {}
        Convert(new_list2, di2)
        del parsed2['parameter']
        del parsed2['value']
        parsed2['values']=di2
        airr= (json.dumps(parsed2, indent=4, sort_keys=True))
        return(airr)
    if(param=="airlight"):
        your_json=(str(js2)+str(js))
        new=your_json.split("}")
        air=new[0]+"}"
        light=new[1]+"}"+"}"
        parsed2 = json.loads(air)
        new_list2=[(parsed2['parameter'],parsed2['value'])]
        di2= {}
        Convert(new_list2, di2)
        del parsed2['parameter']
        del parsed2['value']
        parsed2['values']=di2
        parsed = json.loads(light)
        new_list=[]
        for key, value in parsed["0"].items():
            if(key!="lat" and key!="long"):
                new_list.append((key,parsed["0"][key]))
        di= {}
        Convert(new_list, di)
        for k in list(parsed["0"].keys()):
            if k != 'lat' and k != 'long':
                del parsed["0"][k]
        parsed["lat"]=parsed["0"]["lat"]
        parsed["long"]=parsed["0"]["long"]
        del parsed["0"]
        data = {}
        data.update(di) 
        data.update(di2)
        parsed['values']=data
        return (json.dumps(parsed, indent=4, sort_keys=True))
    #viirs
    req = Request("https://ngdc.noaa.gov/eog/viirs/download_flare_only_iframe.html")
    html_page = urlopen(req)

    soup = BeautifulSoup(html_page, "lxml")

    links = []
    for link in soup.findAll('a'):
        links.append(link.get('href'))

    #print(links)
    sub = 'd201712'
    links_dec=[]
    links_dec=([s for s in links if sub in s])
    #print(links_dec)
    sub = 'only.csv.gz'
    links_dec_csv=[]
    links_dec_csv=([s for s in links_dec if sub in s])
    url =links_dec_csv[0]
    filename = "latest.csv.gz"
    with open(filename, "wb") as f:
        r = requests.get(url)
        f.write(r.content)
    with gzip.open('latest.csv.gz', 'rb') as f_in:
        with open('latest.csv', 'wb') as f_out:
            shutil.copyfileobj(f_in, f_out)
    chunksize = 10 ** 6
    viirs=pd.DataFrame()
    for chunk in pd.read_csv("latest.csv", chunksize=chunksize):
        viirs=viirs.append(chunk)
        
    #search for nearest lat long in the viirsand put as layer
    virlight=gis.content.import_data(viirs)
    virlight.layer.layers = [dict(virlight.layer)]
    map1.add_layer(virlight)
    virdf=virlight.query()
    dfn=((virdf).df.head())
    #try2=dfn.loc[key1]
    #print(try1)
    js3=dfn.to_json(orient='index')
    if(param=='viirs'):
        return(js3)
if __name__=='__main__':
	run_simple('localhost', 80, app)

 * Running on http://localhost:9005/ (Press CTRL+C to quit)
The maximum number of records allowed (1000) has been exceeded.
[2018-06-26 17:14:11,550] ERROR in app: Exception on /display/28.6466773,76.813073,1496275200,viirs [GET]
Traceback (most recent call last):
  File "C:\Users\SLN lab\Anaconda3\lib\site-packages\flask\app.py", line 2292, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\SLN lab\Anaconda3\lib\site-packages\flask\app.py", line 1815, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Users\SLN lab\Anaconda3\lib\site-packages\flask\app.py", line 1718, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "C:\Users\SLN lab\Anaconda3\lib\site-packages\flask\_compat.py", line 35, in reraise
    raise value
  File "C:\Users\SLN lab\Anaconda3\lib\site-packages\flask\app.py", line 1813, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\Users\SLN lab\Anaconda3\lib\site-packages\flask\app.py", lin

In [1]:
import csv
import pandas as pd
chunksize = 10 ** 6
df=pd.DataFrame()
for chunk in pd.read_csv("latest.csv", chunksize=chunksize):
    df=df.append(chunk)
print((df))

        id                              id_Key            Date_Proc  \
0     1418  x0370421W_y050242S_l2010_s2317_v21  2017/12/05 04:45:21   
1        4  x1227004E_y638804N_l0460_s3075_v21  2017/12/05 20:52:26   
2        5  x1227163E_y638712N_l0460_s3076_v21  2017/12/05 20:52:26   
3        8  x1226919E_y638632N_l0461_s3076_v21  2017/12/05 20:52:26   
4       12  x1226934E_y638653N_l0469_s3076_v21  2017/12/05 20:52:26   
5       27  x1128017E_y613517N_l1229_s2974_v21  2017/12/05 20:52:26   
6       28  x1128166E_y613449N_l1229_s2975_v21  2017/12/05 20:52:26   
7       33  x1128237E_y613514N_l1236_s2975_v21  2017/12/05 20:52:26   
8       34  x1128386E_y613448N_l1236_s2976_v21  2017/12/05 20:52:26   
9       36  x1127907E_y613490N_l1237_s2974_v21  2017/12/05 20:52:26   
10      37  x1128057E_y613424N_l1237_s2975_v21  2017/12/05 20:52:26   
11      49  x0579323E_y686141N_l1339_s0128_v21  2017/12/05 20:52:26   
12      50  x0579639E_y686156N_l1339_s0129_v21  2017/12/05 20:52:26   
13    